# Data Science Homework 1

For this homework we'll be looking at the data released by the State Bank of Pakistan. The data is the daily bank-wise and donor-wise receipts of the fund for the Daimer Bhasha and Mohmand Dam. You can find them in the following link: http://www.sbp.org.pk/notifications/FD/DamFund/Damfund.htm. Take a moment to look around the data and try to figure out what the possible challenges could be.

The main purpose of this homework is to teach how to scrape data from the web, clean it, and import it into Pandas for data analysis purposes. There are however some things to note:
1. As you can tell, the data is in PDF form. PDF is the most difficult to handle data format and if you get extremely broken CSV files, there isn't a need to worry, that's where the cleaning part comes in.
2. We'll be using an API to convert the data from PDF to CSV, and then from CSV to Pandas. There are, however, other ways to do this. The reason we wanted to do this method is two-fold
    * It will teach you how to communicate with APIs using Python, which will be a useful skill when you want to deploy your data models as an API so that it can work with other APIs that need those data models. Moreover, a lot of data you get in the real world is from APIs. 
    * The CSV will be extremely inconsistent, so it will give you immense practice with using regular-expressions, which are extremely important in the Data Science tool-kit.
    
Submit the notebooks in a similar format to the Labs: print the relevant output in each cell **only if it has an output. The initial scraping and converting does not have any output**, and name the notebooks as:
**rollnumber_HW1.ipynb** for e.g **20100237_H1.ipynb**

Please make sure you complete full parts (denoted by a Header each in this notebook) as the grading will be based on parts. Needless to say, do not copy someone else's code. In most Data Science careers, the main skill is not how good you are at coding, but how well you are able to use the tools at your disposal and what inferences you are able to make with the information that you have. Thus, while you might be able to do the HW by looking at someone else's code, unless you go through the actual thought process, you won't learn a lot.

We'll be using a lot of libraries in this tutorial, make sure you go through them so you understand what they are used for.

**NOTE: If you are more comfortable doing so, as I am, you can do the assignment on your preferred text editor on simple Python and then write the code neatly in a notebook.** Personally, I find Sublime/Vim easier to use than Jupyter, mostly since a lot of shortcuts there make coding much easier, while here the shortcuts are more about navigation and controlling your cells.

**The homework is to be done in pairs of 2.** 

**Naming convention: rollnumber1_rollnumber2_HW1.ipynb**

Total Marks: 100

## Part 0: Getting the Data

You can have a look at the data through the link given above. Download a few PDF files and go through the data to see what it looks like. How many columns are there, each, in the PDF files? Are there any inconsistencies? Any particular values that pop out that would need to be taken care of later in your cleaning? Think of all these questions when going through the initial PDF because they will prove really helpful when you can not figure out why there are so many "NaN" values in your final DataFrame.

## Part 1: Data Scraping              
Marks: 20

We'll be using what is called the *requests* model to get an HTML page, and then use *BeautifulSoup* to parse that HTML page such that we are able to to derive the appropriate information from it. I recommend you go through the documentation of each to learn more about how to use the libraries. 

* [Requests Documentation](http://docs.python-requests.org/en/master/)
* [BeautifulSoup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
* [BeautifulSoup + Requests Tutorial](https://www.pythonforbeginners.com/python-on-the-web/web-scraping-with-beautifulsoup)
* [BeautifulSoup](https://medium.freecodecamp.org/how-to-scrape-websites-with-python-and-beautifulsoup-5946935d93fe) Note that this tutorial is more detailed. I would highly recommend you go through this as well even though the library used here is urllib2 instead of requests (which you can do as well!). It also links to more web-scraping libraries like Scrapy for more complicated scraping.

In [35]:
import requests
from bs4 import BeautifulSoup
import os

html_url = "http://www.sbp.org.pk/notifications/FD/DamFund/Damfund.htm"
url = "http://www.sbp.org.pk/notifications/FD/DamFund/"

html_page = requests.get(html_url)

html_page.text

soup = BeautifulSoup(html_page.text, 'html.parser')


listOfPdfs=[]

for link in soup.find_all('a'):
    if(link.get('href').startswith("Datewise")):
        listOfPdfs.append(link.get('href'))

for i in range(len(listOfPdfs)):
    listOfPdfs[i] = url + listOfPdfs[i]


In [45]:
for i in range(len(listOfPdfs)):
    nameToSet = (((listOfPdfs[i].split('/'))[-1]).split('.'))[0]

05-10-2018
04-10-2018
03-10-2018
02-10-2018
01-10-2018
28-09-2018
27-09-2018
26-09-2018
25-09-2018
24-09-2018
19-09-2018
18-09-2018
17-09-2018
14-09-2018
13-09-2018
12-09-2018
11-09-2018
10-09-2018
07-09-2018
06-09-2018
05-09-2018
04-09-2018
03-09-2018
31-08-2018
30-08-2018
29-08-2018
28-08-2018
27-08-2018
24-08-2018
20-08-2018
17-08-2018
16-08-2018
15-08-2018
13-08-2018
10-08-2018
09-08-2018
08-08-2018
07-08-2018
06-08-2018
03-08-2018
02-08-2018
01-08-2018
31-07-2018
30-07-2018
27-07-2018
26-07-2018
24-07-2018
23-07-2018
20-07-2018
19-07-2018
18-07-2018
17-07-2018
16-07-2018
13-07-2018
12-07-2018
11-07-2018
10-07-2018
09-07-2018
06-07-2018


In [39]:
newpath = r'./allPdfs'

if not os.path.exists(newpath):
    os.mkdir(newpath)
else:
    print ("Directory already exists")
    
    
for i in range(len(listOfPdfs)):
    addpdf = requests.get(listOfPdfs[i])
    nameToSet = (((listOfPdfs[i].split('/'))[-1]).split('.'))[0]
    with open("allPdfs/"+nameToSet+".pdf", "wb") as f:
        f.write(addpdf.content)

print("Downloaded all Files")


Directory already exists
Downloaded all Files


## Part 2: Converting from PDF to CSV
Marks: 15

You have two possible options between deciding what API to use for the conversion task.

The first option is communicating with an API called [Zamzar](https://www.zamzar.com) to send each PDF, ask them to convert it into CSV, and then download the converted CSV. They provide sample code to do everything from generating a simple request to starting a conversion job, checking for completion, and then downloading the finished file. You can find this information on the [Zamzar Documentation](https://developers.zamzar.com/docs) page.

**Important Information: **

The API only provides 100 points of free conversion, and each PDF to CSV conversion costs 3 points, that means with one account you can only convert **33** PDFs. However, this also means you have very little room to play around with this API, unless you have an extra email-address, so you need to be very careful when coding to communicate with this API. 

Moreover, the API only keeps the converted files for one day with a free account, so make sure you do this part in one go.

**Note: Using the Zamzar API grants a bonus of 10 marks. This will help if you are not able to complete this assignment, or it can be used up in a later assignment if you get 110/100 marks in this one.**

Another extremely simple API is the [PDF Tables](https://pdftables.com) API which is much simpler to use than the Zamzar API, however does not allow you to check the job for completion or for any intermediate steps. Moreover, this requires the installation of a library. Once again, they allow only 50 versions for free, but that is enough conversions for us. This [blog post](https://pdftables.com/blog/pdf-to-excel-with-python) will help you figure out how to convert the PDF to CSV using Python.

The cons of this API is that it will not really teach you any proper API communcation through requests since you do not have to navigate through any requests.

In [40]:
from requests.auth import HTTPBasicAuth as HBA
import glob
from time import sleep

1.248651

In [41]:
pdfs_folder = './allPdfs/*.pdf'


job_ids = []

bakar_api = "1ebe14564fe42d5cd16185dc688247add9e1226a"
hadi_api = "2c534c54fcf004bae076553fac29837311c24c3b"
endpoint = "https://sandbox.zamzar.com/v1/jobs"


target_format = "csv"

i = 0

for file_name in glob.glob(pdfs_folder):
    source_file = file_name
    file_content = {'source_file': open(source_file, 'rb')}
    data_content = {'target_format': target_format}
    if((os.path.getsize(file_name)/1000000) < 1.0 ):
        res = requests.post(endpoint, data=data_content, files=file_content, auth=HBA(bakar_api, ''))
        job_ids.append(res.json()['id'])
    if(i == 32): 
        bakar_api = hadi_api
    i = i + 1
    
    
for i in range(len(job_ids)):
    print("ID: ", job_ids[i])

ID:  3979788
ID:  3979789
ID:  3979790
ID:  3979791
ID:  3979792
ID:  3979794
ID:  3979796
ID:  3979797
ID:  3979799
ID:  3979801
ID:  3979803
ID:  3979804
ID:  3979805
ID:  3979806
ID:  3979807
ID:  3979808
ID:  3979809
ID:  3979810
ID:  3979811
ID:  3979813
ID:  3979814
ID:  3979815
ID:  3979816
ID:  3979817
ID:  3979819
ID:  3979820
ID:  3979821
ID:  3979823
ID:  3979824
ID:  3979827
ID:  3979829
ID:  3979831
ID:  3979832
ID:  3979833
ID:  3979836
ID:  3979837
ID:  3979838
ID:  3979840
ID:  3979841
ID:  3979843
ID:  3979844
ID:  3979846
ID:  3979847
ID:  3979848
ID:  3979850
ID:  3979851
ID:  3979852
ID:  3979853
ID:  3979854
ID:  3979855
ID:  3979856
ID:  3979857
ID:  3979858
ID:  3979859
ID:  3979860
ID:  3979861
ID:  3979862
ID:  3979863


Below this cell write the code to download the completed files. First check if a job_id's status is completed and wait until it is. After it has been completed, download the file and save it.

The exact code required here is all in the documentation, the only additional task you have to do on your own is figure out a way to find out which file has just been received from the job_id, and name the local file.

**Please look at the Example JSON response in the [documentation](https://developers.zamzar.com/docs) to learn how to figure out the filenames, job status etc**

In [43]:
## Your code goes here ##
newpath = r'./allCSVs'

bakar_api = "1ebe14564fe42d5cd16185dc688247add9e1226a"
hadi_api = "2c534c54fcf004bae076553fac29837311c24c3b"
api_key = bakar_api

if not os.path.exists(newpath):
    os.mkdir(newpath)
else:
    print ("Directory already exists")

for i in range(len(job_ids)):
    if(i > 32):
        api_key = hadi_api
    while(1):
        sleep(3)
        job_id = job_ids[i]
        endpoint2 = "https://sandbox.zamzar.com/v1/jobs/{}".format(job_id)
        res2 = requests.get(endpoint2, auth=HTTPBasicAuth(api_key, ''))
        if(res2.json()['finished_at'] != None ):
            file_id = ((res2.json()['target_files'])[0])['id']
            endpoint3 = "https://sandbox.zamzar.com/v1/files/{}/content".format(file_id)
            resp1 = requests.get(endpoint3, stream = True , auth=HBA(api_key, ''))
            filetodownload = (((res2.json()['source_file'])['name']).split('.'))[0] + ".csv"
            
            try:
                with open("allCSVs/"+filetodownload, 'wb') as f:
                    for chunk in resp1.iter_content(chunk_size=1024):
                        if chunk:
                            f.write(chunk)
                            f.flush()

                    print(filetodownload, " downloaded")
                    break
            except IOError:
                print("error")

01-08-2018.csv  downloaded
01-10-2018.csv  downloaded
02-08-2018.csv  downloaded
02-10-2018.csv  downloaded
03-08-2018.csv  downloaded
03-09-2018.csv  downloaded
03-10-2018.csv  downloaded
04-09-2018.csv  downloaded
04-10-2018.csv  downloaded
05-09-2018.csv  downloaded
05-10-2018.csv  downloaded
06-07-2018.csv  downloaded
06-08-2018.csv  downloaded
06-09-2018.csv  downloaded
07-08-2018.csv  downloaded
07-09-2018.csv  downloaded
08-08-2018.csv  downloaded
09-07-2018.csv  downloaded
09-08-2018.csv  downloaded
10-07-2018.csv  downloaded
10-08-2018.csv  downloaded
10-09-2018.csv  downloaded
11-07-2018.csv  downloaded
11-09-2018.csv  downloaded
12-07-2018.csv  downloaded
12-09-2018.csv  downloaded
13-07-2018.csv  downloaded
13-08-2018.csv  downloaded
13-09-2018.csv  downloaded
14-09-2018.csv  downloaded
15-08-2018.csv  downloaded
16-07-2018.csv  downloaded
16-08-2018.csv  downloaded
17-07-2018.csv  downloaded
17-08-2018.csv  downloaded
17-09-2018.csv  downloaded
18-07-2018.csv  downloaded
1

## Part 3: Parsing the CSV File
Marks: 35

This is perhaps the most difficult part of the assignment, you have to follow a similar strategy to what you did the Udacity Lab 1. You can not simply use Pandas read_csv since the conversion is not perfect and there will be rows with different number of columns, which Pandas does not take care of.

### **Main Task:**
* Write a function that parses a CSV into a Pandas DataFrame
* Each DataFrame should consist of three columns with headers Bank, Donor_Name, and Amount
* The date should be retrieved from the given filename 
* The Donor_Name can be NaN, as it is in a lot of cases. But try to retrieve as much information as possible
* Remove all "Page of" rows
* Don't include the header rows (e.g. "SUPREME COURT FUND....") into the DataFrame
* The Amount should be converted into a Pandas numeric at the end

### Other info:
Some important resources for this part are (you can choose any one tutorial that you feel is easy to understand, they all cover roughly the same content):
* [RegEx Tutorial 1](https://www.regular-expressions.info/)
* [RegEx Tutorial 2](https://regexone.com/lesson/introduction_abcs)
* [RegEx Tutorial 3](https://www.rexegg.com/)
* [RegEx Cheatsheat](https://medium.com/factory-mind/regex-tutorial-a-simple-cheatsheet-by-examples-649dc1c3f285)
* This [RegEx Editor](https://regex101.com/) is your best friend since you can test your expression separately on this

You will probably have to use the CSV reader in order to get all the rows of the file. You can learn more about it using this [tutorial](https://www.alexkras.com/how-to-read-csv-file-in-python/).

Some tips:
* First find out how many columns are in each row
* Print out rows which are longer than they should be (they should all be of length 3)
* Try to find patterns in how the data is spread, and what common problems exist in all rows
* Write some regex to try an extract the amount from the problem row and then:
    * Put the amount as the third column
    * Merge the rest of the string as a name of the donor in the 2nd column
* Also check if the rows with 3 columns are correctly formatted or not, many of them would probably not be.

In [3]:
import csv
import re # To use regular expressions
import pandas as pd
import glob


currency4 = re.compile('(,*\d{1,3},\d{3}[,\d{3}]*|,\d{1,3}[^\d])')
currency5 = re.compile('\d{1,3}')
numexp = re.compile('\"?\s*(\d[\d|,]*\.?\d{1,2})\s*\"?|\"?\s*(\d)\s*\"?')


# example_file = './allCSVs/01-08-2018.csv' # Assuming the file is in the folder all_csvs and is named appropriately
# This is one of the most problematic files which is why I have included this in the example

In [4]:
def test(filename):
    data = []
    li = []
    row1 = []
    with open(filename) as f:
        reader = csv.reader(f)
        next(reader) # skip header
        for row in reader:
            li.append(row)
            row1.append(row[0])
            row1.append(",".join(row[1:]))
            data.append(row1)
            row1 = []
        return li,data

In [5]:
def giveAmount(vals):
#     print(vals)
    am = ""
    if(len(vals)<4):
        am = currency5.findall(vals)
    else:
        am = currency4.findall(vals)
    new_am = ""
    if(len(am) == 1):
        if(vals[vals.find(am[0]) - 1].isdigit() == True and vals[vals.find(am[0])] != ','):
            new_am = "Nan"
            return(new_am, '')
        else:
            new_am = am[0].replace(',', '').replace('/','')
            if(len(new_am) <= 9):
                return(new_am, am[0])
            else:
                new_am = "Nan"
                return(new_am, '')
                
    else:
        new_am = "Nan"
        return (new_am, '')
        
def giveAandR(remaining):
    matlooba = numexp.findall(remaining)
    newxyz = ""
    for i in matlooba:
        newxyz += i[0]+"/"
    return giveAmount(newxyz)


def retNameAndAmount(val):
    orig = val
    amountFinal, toRemove = giveAandR(val)
    orig = orig.replace(toRemove, '')
    return orig, amountFinal

In [6]:
def funcall(filename):
    resultList,resultWord = test(filename)

    true = 0
    data = resultWord
    dataForDataFrame = []
    index = []
    total = 0

    lis = filename.split('\\')
    date = lis[-1].split('.')[0]

    for a in data:
        if(a == ['Bank', 'Depositor Name,Amount']):
            true = 1
            continue
        if(true == 1):
            bankname = a[0]
            if(len(bankname) != 0):
                last_word = (bankname.split())[-1]
                first_word = (bankname.split())[0]
                if(last_word == 'Total'):
                    continue
                if(first_word == "Page" or first_word =="page" or first_word == "PAGE"):
                    continue
                else:
                    remaining = a[1]
                    dataitem = []
                    dataitem.append(bankname)
                    name,amount = retNameAndAmount(remaining)
                    dataitem.append(name)
                    dataitem.append(pd.to_numeric(amount, errors="coerce"))
                    dataitem.append(date)
                    dataForDataFrame.append(dataitem)
                    total+=1
                    index.append(total)
            else:
                continue

    return (dataForDataFrame,index)
    

            


In [7]:
# Remember, remove headers and convert all amounts to Numeric; if it can't be converted it needs to be NaN
def read_csv(filename):
#     print(filename)
    filaname = filename.replace('\\', '/')
    raw_headers = ['Bank', 'Donor Name','Amount','Date']
    
    raw_data, raw_indexes = funcall(filename)
    
    df = pd.DataFrame(raw_data,raw_indexes,raw_headers)
    
    return df
    
# print(read_csv(example_file))

## Part 4: Importing Full Dataset
Marks: 10 

The only additional task in this part is to:
* Run the parser on all the files
* For each file **add a 'Date' column, which should be inferred from the filename**
* Concatenate each DataFrame into one large DataFrame. *Hint: concat*

In [10]:
files = glob.glob('./allCSVs/*.csv')

dataf1 = []
dataf2 = []

dataf1 = read_csv(files[0])
full_data = read_csv(files[1])

full_data = pd.concat([dataf1, full_data], ignore_index=True)

for i in range(2, len(files)):
    dataf1 = read_csv(files[i])
    full_data = pd.concat([dataf1, full_data], ignore_index=True)
    
# pd.DataFrame(dataf1)





full_data

# print (full_data.head())
# print (full_data.shape)
# print (full_data.tail())

,Bank,Donor Name,Amount,Date
0,SBP BSC,"SALMA HAMID,",100000.0,31-08-2018
1,SBP BSC,"TEHREEM SUBHAN,",4000.0,31-08-2018
2,SBP BSC,"AMINA NAZ,",1000.0,31-08-2018
3,SBP BSC,"HASSAN MUNIR,500",NaN,31-08-2018
4,SBP BSC,"IMAM BAKHSH,200",NaN,31-08-2018
5,SBP BSC,"FIRST MICRO FINANCE BANK,",1110.0,31-08-2018
6,SBP BSC,"GC UNIVERSITY FSD,",2398014.0,31-08-2018
7,SBP BSC,"PAKISTAN MISSION UK,",2683838.0,31-08-2018
8,SBP BSC,"MULTIPLE DONORS,100",NaN,31-08-2018
9,SBP BSC,"SALAH UD DIN,",35770.0,31-08-2018


## Part 5: Data Integrity Checks
Marks: 20

* How many NaN values are there in each column? Why are they there? 
* What are the maximum and minimum values, is there anything peculiar about the max values?
* Are there any rows which are not NaN but should still be a different DataFrame altogether?
* Should these problem rows be removed? Can they be useful in other ways?

In [11]:
full_data.isna().sum()


# Number of NaN values = 50691.
# They are there because the csv files are not propperly formatted and there are many different kinds of values which makes
# difficult to extract the amount from the rows. For example, mobile numbers and amounts are written together and we cannot
# be sure from where exactly the amount value starts from the numbers

Bank              0
Donor Name        0
Amount        50691
Date              0
dtype: int64

In [18]:
full_data.max()

full_data.iloc[full_data['Amount'].idxmax()]

#Pakistan Army gave the maximum amount which is displayed when the cell is run

Bank               SBP BSC
Donor Name    PAK ARMY,.00
Amount         5.82072e+08
Date            11-09-2018
Name: 106680, dtype: object

In [19]:
full_data.iloc[full_data['Amount'].idxmin()]

#Minimum Value

Bank             United Bank Limited
Donor Name    ADNAN ALI TID 11183900
Amount                             1
Date                      30-07-2018
Name: 9245, dtype: object

# 3 and 4

* Are there any rows which are not NaN but should still be a different DataFrame altogether?

In the bankname and donor name columns there are alot of values which mean nothing. These are because in some rows in the data there are values without names or amounts.
* Should these problem rows be removed? Can they be useful in other ways?

Yes these problem rows should be removed. Data should be cleaned as much as possible to ensure data integrity and hence the accuracy of the results that we use in data analysis.